In [8]:
import numpy as np
import librosa as lb # for reference, my solutions use librosa version 0.7.2
import matplotlib.pyplot as plt
from tqdm import tqdm
from timeit import default_timer as timer
from numba import jit

In [2]:
%matplotlib inline
%load_ext Cython

### NWTW Implementation

In [4]:
@jit(nopython=True)
def nwtw_backtrace(D, B, steps):
    '''
    Backtraces through the cumulative cost matrix D.
    
    Arguments:
    D -- cumulative cost matrix
    B -- backtrace matrix
    steps -- a numpy matrix specifying the allowable transitions.  It should be of
            dimension (L, 2), where each row specifies (row step, col step)
    
    Returns:
    path -- a python list of (row, col) coordinates for the optimal path.
    '''

    path = []

    pos = (D.shape[0] - 1, D.shape[1] - 1)
    path.append(pos)
    while (pos != (0,0)):
        (row, col) = pos
        stepidx = B[row, col]
        (rstep, cstep) = steps[stepidx]
        pos = (row-rstep, col-cstep)
        path.append(pos)
    
    return path

In [11]:
@jit(nopython=True)
def NWTW_faster(C, gamma=0.346):
    D = np.zeros(C.shape)
    B = np.zeros(C.shape, dtype=np.int8)

    # Allowed steps as specified by the paper
    steps = np.array([0, 1, 1, 0, 1, 1, 1, 2, 2, 1]).reshape((5,2))

    # initialize -- makes inner for loop more efficient
    D[:,0] = gamma * np.arange(C.shape[0])
    B[:,0] = 1
    D[0,:] = gamma * np.arange(C.shape[1])
    B[0,:] = 0
    D[0,0] = 0 # in case gamma = np.inf

    for i in range(1, C.shape[0]):
        for j in range(1, C.shape[1]):
            # Min bracket
            mincost = np.inf
            minidx = -1
            for stepidx, step in enumerate(steps):
                (rstep, cstep) = step
                prevrow = i - rstep
                prevcol = j - cstep
                if prevrow >= 0 and prevcol >= 0:
                    pathcost = D[prevrow, prevcol]
                    # nw(i - 1, j - 2)
                    if (stepidx == 3):
                        pathcost += C[i, j - 1] + C[i, j]
                    # nw(i - 2, j - 1)
                    elif (stepidx == 4):
                        pathcost += C[i - 1, j] + C[i, j]
                    # nw(i - 1, j - 1)
                    elif (stepidx == 2):
                        pathcost += C[i, j]
                    else:
                        pathcost += gamma
                    if pathcost < mincost:
                        mincost = pathcost
                        minidx = stepidx
            # TT: is this if clause necessary with the for loops starting from 1?  I think it can be removed
            if minidx == -1:
                raise Exception("NWTW edge case error with -1 indexes")
            D[i, j] = mincost
            B[i, j] = minidx

    optcost = D[-1,-1]
    path = nwtw_backtrace(D, B, steps)
    path.reverse()

    path = np.array(path).T

    return optcost, path, D, B

# CPython Implementation

In [12]:
# %%cython
# import numpy as np
# cimport numpy as np
# cimport cython

# import sys
# import time


# DTYPE_INT32 = np.int32
# ctypedef np.int32_t DTYPE_INT32_t

# DTYPE_FLOAT = np.float64
# ctypedef np.float64_t DTYPE_FLOAT_t

# cdef DTYPE_FLOAT_t MAX_FLOAT = float('inf')

# # careful, without bounds checking can mess up memory - also can't use negative indices I think (like x[-1])
# @cython.boundscheck(False) # turn off bounds-checking for entire function
# def NWTW_Cost_To_AccumCostAndSteps(Cin, parameter, gamma):
#     '''
#     Inputs
#         C: The cost Matrix
#     '''

#     '''
#     Section for checking and catching errors in the inputs
#     '''

#     cdef np.ndarray[DTYPE_FLOAT_t, ndim=2] C
#     try:
#         C = np.array(Cin, dtype=DTYPE_FLOAT)
#     except TypeError:
#         print(bcolors.FAIL + "FAILURE: The type of the cost matrix is wrong - please pass in a 2-d numpy array" + bcolors.ENDC)
#         return [-1, -1, -1]
#     except ValueError:
#         print(bcolors.FAIL + "FAILURE: The type of the elements in the cost matrix is wrong - please have each element be a float (perhaps you passed in a matrix of ints?)" + bcolors.ENDC)
#         return [-1, -1, -1]

#     cdef np.ndarray[np.uint32_t, ndim=1] dn
#     cdef np.ndarray[np.uint32_t, ndim=1] dm
#     # make sure dn, dm, and dw are setup
#     # dn loading and exception handling
#     if ('dn'  in parameter.keys()):
#         try:
#             dn = np.array(parameter['dn'], dtype=np.uint32)
#         except TypeError:
#             print(bcolors.FAIL + "FAILURE: The type of dn (row steps) is wrong - please pass in a 1-d numpy array that holds uint32s" + bcolors.ENDC)
#             return [-1, -1, -1]
#         except ValueError:
#             print(bcolors.FAIL + "The type of the elements in dn (row steps) is wrong - please have each element be a uint32 (perhaps you passed a long?). You can specify this when making a numpy array like: np.array([1,2,3],dtype=np.uint32)" + bcolors.ENDC)
#             return [-1, -1, -1]
#     else:
#         dn = np.array([1, 1, 0], dtype=np.uint32)
#     # dm loading and exception handling
#     if 'dm'  in parameter.keys():
#         try:
#             dm = np.array(parameter['dm'], dtype=np.uint32)
#         except TypeError:
#             print(bcolors.FAIL + "FAILURE: The type of dm (col steps) is wrong - please pass in a 1-d numpy array that holds uint32s" + bcolors.ENDC)
#             return [-1, -1, -1]
#         except ValueError:
#             print(bcolors.FAIL + "FAILURE: The type of the elements in dm (col steps) is wrong - please have each element be a uint32 (perhaps you passed a long?). You can specify this when making a numpy array like: np.array([1,2,3],dtype=np.uint32)" + bcolors.ENDC)
#             return [-1, -1, -1]
#     else:
#         print(bcolors.FAIL + "dm (col steps) was not passed in (gave default value [1,0,1]) " + bcolors.ENDC)
#         dm = np.array([1, 0, 1], dtype=np.uint32)

    
#     '''
#     Section where types are given to the variables we're going to use 
#     '''
#     # create matrices to store our results (D and E)
#     cdef DTYPE_INT32_t numRows = C.shape[0] # only works with np arrays, use np.shape(x) will work on lists? want to force to use np though?
#     cdef DTYPE_INT32_t numCols = C.shape[1]
#     cdef DTYPE_INT32_t numDifSteps = np.size(dn)

#     cdef unsigned int maxRowStep = max(dn)
#     cdef unsigned int maxColStep = max(dm)

#     cdef np.ndarray[np.uint32_t, ndim=2] steps = np.zeros((numRows,numCols), dtype=np.uint32)
#     cdef np.ndarray[DTYPE_FLOAT_t, ndim=2] accumCost = np.ones((maxRowStep + numRows, maxColStep + numCols), dtype=DTYPE_FLOAT) * MAX_FLOAT

#     cdef DTYPE_FLOAT_t bestCost
#     cdef DTYPE_INT32_t bestCostIndex
#     cdef DTYPE_FLOAT_t costForStep
#     cdef unsigned int row, col
#     cdef unsigned int stepIndex

#     '''
#     The start of the actual algorithm, now that all our variables are set up
#     '''
#     # initializing the cost matrix - depends on whether its subsequence DTW
#     # essentially allow us to hop on the bottom anywhere (so could start partway through one of the signals)

#     accumCost[2:,maxColStep] = gamma * np.arange(C.shape[0], dtype=DTYPE_INT32)
#     steps[:,0] = 1
#     accumCost[maxRowStep,2:] = gamma * np.arange(C.shape[1], dtype=DTYPE_INT32)
#     steps[0,:] = 0
#     accumCost[maxRowStep,maxColStep] = 0 # in case gamma = np.inf

#     # filling the accumulated cost matrix
#     for row in range(maxRowStep + 1, numRows + maxRowStep, 1):
#         for col in range(maxColStep + 1, numCols + maxColStep, 1):
#             bestCost = accumCost[<unsigned int>row, <unsigned int>col] # initialize with what's there - so if is an entry point, then can start low
#             bestCostIndex = 0
#             # go through each step, find the best one
#             for stepIndex in range(numDifSteps):
#                 costForStep = accumCost[<unsigned int>((row - dn[(stepIndex)])), <unsigned int>((col - dm[(stepIndex)]))]
#                 # nw(i - 1, j - 2)
#                 if (stepIndex == 3):
#                     costForStep += C[<unsigned int>(row - maxRowStep), <unsigned int>(col - maxColStep - 1)] + C[<unsigned int>(row - maxRowStep), <unsigned int>(col - maxColStep)]
#                 # nw(i - 2, j - 1)
#                 elif (stepIndex == 4):
#                     costForStep += C[<unsigned int>(row - maxRowStep - 1), <unsigned int>(col - maxColStep)] + C[<unsigned int>(row - maxRowStep), <unsigned int>(col - maxColStep)]
#                 # nw(i - 1, j - 1)
#                 elif (stepIndex == 2):
#                     costForStep += C[<unsigned int>(row - maxRowStep), <unsigned int>(col - maxColStep)]
#                 else:
#                     costForStep += gamma

#                 if costForStep < bestCost:
#                     bestCost = costForStep
#                     bestCostIndex = stepIndex
#             # save the best cost and best cost index
#             accumCost[<unsigned int>(row), <unsigned int>(col)] = bestCost
#             steps[<unsigned int>(row - maxRowStep), <unsigned int>(col - maxColStep)] = bestCostIndex
#     # return the accumulated cost along with the matrix of steps taken to achieve that cost
#     return [accumCost[maxRowStep:, maxColStep:], steps]

# @cython.boundscheck(False) # turn off bounds-checking for entire function
# def NWTW_GetPath(np.ndarray[DTYPE_FLOAT_t, ndim=2] accumCost, np.ndarray[np.uint32_t, ndim=2] stepsForCost, parameter):
#     '''

#     Parameter should have: 'dn', 'dm', 'dw', 'SubSequence'
#     '''

#     cdef np.ndarray[unsigned int, ndim=1] dn
#     cdef np.ndarray[unsigned int, ndim=1] dm
#     cdef np.uint8_t subseq
#     cdef np.int32_t startCol # added
#     # make sure dn, dm, and dw are setup
#     if ('dn'  in parameter.keys()):
#         dn = parameter['dn']
#     else:
#         dn = np.array([1, 1, 0], dtype=DTYPE_INT32)
#     if 'dm'  in parameter.keys():
#         dm = parameter['dm']
#     else:
#         dm = np.array([1, 0, 1], dtype=DTYPE_INT32)
#     subseq = 0
    

#     cdef np.uint32_t numRows
#     cdef np.uint32_t numCols
#     cdef np.uint32_t curRow
#     cdef np.uint32_t curCol
#     cdef np.uint32_t endCol
#     cdef DTYPE_FLOAT_t endCost

#     numRows = accumCost.shape[0]
#     numCols = accumCost.shape[1]

#     # either start at the far corner (non sub-sequence)
#     # or start at the lowest cost entry in the last row (sub-sequence)
#     # where all of the signal along the row has been used, but only a 
#     # sub-sequence of the signal along the columns has to be used
#     curRow = numRows - 1
#     curCol = numCols - 1

#     endCol = curCol
#     endCost = accumCost[curRow, curCol]

#     cdef np.uint32_t curRowStep
#     cdef np.uint32_t curColStep
#     cdef np.uint32_t curStepIndex


#     cdef np.ndarray[np.uint32_t, ndim=2] path = np.zeros((2, numRows + numCols), dtype=np.uint32) # make as large as could need, then chop at the end
#     path[0, 0] = curRow
#     path[1, 0] = curCol

#     cdef np.uint32_t stepsInPath = 1 # starts at one, we add in one before looping
#     cdef np.uint32_t stepIndex = 0
#     cdef np.int8_t done = (subseq and curRow == 0) or (curRow == 0 and curCol == 0)
#     while not done:
#         if accumCost[curRow, curCol] == MAX_FLOAT:
#             print('A path is not possible')
#             break

#         # you're done if you've made it to the bottom left (non sub-sequence)
#         # or just the bottom (sub-sequence)
#         # find the step size
#         curStepIndex = stepsForCost[curRow, curCol]
#         curRowStep = dn[curStepIndex]
#         curColStep = dm[curStepIndex]
#         # backtrack by 1 step
#         curRow = curRow - curRowStep
#         curCol = curCol - curColStep
#         # add your new location onto the path
#         path[0, stepsInPath] = curRow
#         path[1, stepsInPath] = curCol
#         stepsInPath = stepsInPath + 1
#         # check to see if you're done
#         done = (subseq and curRow == 0) or (curRow == 0 and curCol == 0)

#     # reverse the path (a matrix with two rows) and return it
#     return [np.fliplr(path[:, 0:stepsInPath]), endCol, endCost]

# class bcolors:
#     HEADER = '\033[95m'
#     OKBLUE = '\033[94m'
#     OKGREEN = '\033[92m'
#     WARNING = '\033[93m'
#     FAIL = '\033[91m'
#     ENDC = '\033[0m'
#     BOLD = '\033[1m'
#     UNDERLINE = '\033[4m'

In [ ]:
def L2norm(F):
    L2norm = np.sqrt(np.sum(F*F, axis = 0)) + 1e-9
    Fnorm = F / L2norm.reshape((1,-1))
    return Fnorm

In [1]:
# def alignNWTW(C , downsample, gamma=0.346, profile = False):

#     # Allowed steps as specified by the paper
#     steps = np.array([0, 1, 1, 0, 1, 1, 1, 2, 2, 1]).reshape((5,2))

#     dn = steps[:,0].astype(np.uint32)
#     dm = steps[:,1].astype(np.uint32)
#     parameters = {'dn': dn, 'dm': dm}
#     [D, s] = NWTW_Cost_To_AccumCostAndSteps(C, parameters, gamma)
#     # times.append(time.time())
#     [wp, endCol, endCost] = NWTW_GetPath(D, s, parameters)
#     # times.append(time.time())

# #     if outfile:
# #         pickle.dump(wp, open(outfile, 'wb'))
#     return wp
    
# #     if profile:
# #         return wp, np.diff(times)
# #     else:
# #         return wp

Testing CPython implementation

In [14]:
# C = np.random.randint(5, size=(10000,10000))
# start_p = timer()
# opcost, optath, D_1, B = NWTW_faster(C, gamma = 2)
# end_p = timer()
# steps = np.array([0, 1, 1, 0, 1, 1, 1, 2, 2, 1]).reshape((5,2))
# dn = steps[:,0].astype(np.uint32)
# dm = steps[:,1].astype(np.uint32)
# parameters = {'dn': dn, 'dm': dm}
# start_c = timer()
# [D_2,s] = NWTW_Cost_To_AccumCostAndSteps(C, parameters, gamma = 2)
# end_c = timer()
# assert(np.array_equal(D_1, D_2))
# assert(np.array_equal(s,B))
# print("Matrixes are equal")
# print("Python time on 10000x10000:", end_p - start_p)
# print("CPython time on 10000x10000:", end_c - start_c)


100%|██████████| 9999/9999 [16:07<00:00, 10.34it/s]


Matrixes are equal
Python time on 10000x10000: 967.2974507110193
CPython time on 10000x10000: 7.869221848901361


In [9]:
# F1 = np.random.randint(5, size=(5,5))
# F2 = np.random.randint(5, size=(5,5))
# wp = alignNWTW(F1, F2, downsample=1, gamma = 2)